Import Necessary Libraries 

In [ ]:
!pip install parsivar

     |████████████████████████████████| 36.2MB 102kB/s 
     |████████████████████████████████| 1.5MB 44.0MB/s 
  Created wheel for parsivar: filename=parsivar-0.2.3-cp37-none-any.whl size=36492958 sha256=52f3021cfa9d1013b25de355b0679435553f8583652c1a192c1bf290d1b4eed2
  Stored in directory: /root/.cache/pip/wheels/b4/56/cd/29a045228c81c482429c711646af316ada1218f3029edce37c
  Created wheel for nltk: filename=nltk-3.4.5-cp37-none-any.whl size=1449907 sha256=e1e4a9db98777da1595eb03f45a85768ae2482dbcc65bb1ff4884f12abe04bcc
  Stored in directory: /root/.cache/pip/wheels/96/86/f6/68ab24c23f207c0077381a5e3904b2815136b879538a24b483
Successfully built parsivar nltk
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [ ]:
!pip install langdetect

     |████████████████████████████████| 983kB 4.2MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.8-cp37-none-any.whl size=993193 sha256=50bc6a4147d8ad37c1812de9b4be0b588aee7716ff04b081b36c87b1d8e9951e
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d40a60ea06e6d669e8e5012e6c8b947a57
Successfully built langdetect


In [ ]:
!pip install finglish

In [ ]:
import pandas as pd
from finglish import f2p
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
from google.colab import drive
import os
import sys
from IPython.display import clear_output
from langdetect import detect, DetectorFactory
from parsivar import Normalizer
from parsivar import FindStems
from parsivar import Tokenizer
from parsivar import POSTagger
from string import punctuation
import gensim
import seaborn as sns
import statistics
from collections import Counter
import math
from math import inf
import pickle
import json
import ast
import re
import string
from math import log2

In [ ]:
# you should update this cell according to your settings
config = {
    "current_working_dir" : "/content/drive/MyDrive/NLP-Spring 99-00/HW/HW1/",
    "train_data_path" : "/content/drive/MyDrive/NLP-Spring 99-00/HW/HW1/train_data/train/",
    "k" : 10
}

In [ ]:
drive.mount('/content/drive')

In [ ]:
normalizer = Normalizer()
tokenizer = Tokenizer()
stemmer = FindStems()
tagger = POSTagger(tagging_model="stanford")
alphabet = ""
with open(config["current_working_dir"] + "persian_alphabet.txt", 'r' , encoding='utf-8') as alphabet_file:
    alphabet = alphabet_file.read().replace("\n" , "").replace("\t" , "")
numbers = "0123456789"

Mount Drive

Unzip Data

In [ ]:
import zipfile
if os.path.isdir(config['current_working_dir'] + "train_data") == False:
    print("Unzipping .............." , "\r")
    with zipfile.ZipFile(config['current_working_dir'] + "train.zip","r") as zip_ref:
        zip_ref.extractall(config['current_working_dir'] + "train_data")
    print("Operation was successful")
else:
    print("Folder currently exists!")

Unzipping .............. 
Operation was successful


Read Files

In [ ]:
def read_files(folderpath , gender):
    data = pd.DataFrame(columns = ['file_path' , 'file_text' , 'gender'])
    list_of_all_files = os.listdir(folderpath)
    for filename in list_of_all_files:
        current_file_path = folderpath + "/" + filename
        current_file_text = ""
        with open(current_file_path , 'r' , encoding='utf-8') as current_file:
             current_file_text = current_file.read()
        
        data = data.append({"file_path" : current_file_path , "file_text" : current_file_text , "gender" : gender}, ignore_index=True)
    return data

In [ ]:
csv_path =config['current_working_dir'] + 'raw_train_df.csv'
if os.path.isfile(csv_path):
    data = pd.read_csv(csv_path)
else:
    female_folder = config['train_data_path'] + "female"
    male_folder = config['train_data_path'] + "male"
    data = pd.DataFrame(columns = ['file_path' , 'file_text' , 'gender'])
    data = data.append(read_files(female_folder , 'f'),ignore_index=True)
    data = data.append(read_files(male_folder , 'm'),ignore_index=True)
    data.to_csv(csv_path , index=False)

In [ ]:
data.head()

,file_path,file_text,gender
0,/content/drive/MyDrive/NLP-Spring 99-00/HW1/tr...,معایب این هتل فقط به کارکنان آنهابستگی دارد، و...,f
1,/content/drive/MyDrive/NLP-Spring 99-00/HW1/tr...,من دوباراون هم دی ماه 85و87 به این هتل سفرداشت...,f
2,/content/drive/MyDrive/NLP-Spring 99-00/HW1/tr...,مکان هتل آرامیس بسیار عالی بود دسترسی به اکثر ...,f
3,/content/drive/MyDrive/NLP-Spring 99-00/HW1/tr...,اقا امیر می تونم بپرسم وضعیت غذای هتل از نظر ک...,f
4,/content/drive/MyDrive/NLP-Spring 99-00/HW1/tr...,خب من ابن تابستون بودم و می خوام بگم خیلی بد ب...,f


In [ ]:
data['normalized_text'] = data['file_text'].apply(lambda text : normalizer.normalize(text))

In [ ]:
data['cleaned_number_text'] = data['normalized_text'].apply(lambda text : re.sub(r'\d+', '', text))

In [ ]:
data['cleaned_punc_text'] = data['cleaned_number_text'].apply(lambda text : text.translate(str.maketrans('','', string.punctuation)))

In [ ]:
data['tokenized_sentence'] = data['normalized_text'].apply(lambda text : tokenizer.tokenize_sentences(text))

In [ ]:
def tokenize_words(text):
    if detect(text) != 'fa' and detect(text) != 'ar':
        #convert finglish words to farsi
        tokens = list(map(lambda word : f2p(word) , tokenizer.tokenize_words(text)))
        return tokens
    else:
        return tokenizer.tokenize_words(text)
data['tokenized_word'] = data['cleaned_punc_text'].apply(lambda text : tokenize_words(text))

In [ ]:
data.head()

,file_path,file_text,gender,normalized_text,cleaned_number_text,cleaned_punc_text,tokenized_sentence,tokenized_word
0,/content/drive/MyDrive/NLP-Spring 99-00/HW1/tr...,معایب این هتل فقط به کارکنان آنهابستگی دارد، و...,f,معایب این هتل فقط به کارکنان آنهابستگی دارد ، ...,معایب این هتل فقط به کارکنان آنهابستگی دارد ، ...,معایب این هتل فقط به کارکنان آنهابستگی دارد ، ...,[معایب این هتل فقط به کارکنان آنهابستگی دارد ،...,"[معایب, این, هتل, فقط, به, کارکنان, آنهابستگی,..."
1,/content/drive/MyDrive/NLP-Spring 99-00/HW1/tr...,من دوباراون هم دی ماه 85و87 به این هتل سفرداشت...,f,من دوباراون هم دی ماه 85و87 به این هتل سفرداشت...,من دوباراون هم دی ماه و به این هتل سفرداشتم . ...,من دوباراون هم دی ماه و به این هتل سفرداشتم ا...,[من دوباراون هم دی ماه 85و87 به این هتل سفرداش...,"[من, دوباراون, هم, دی, ماه, و, به, این, هتل, س..."
2,/content/drive/MyDrive/NLP-Spring 99-00/HW1/tr...,مکان هتل آرامیس بسیار عالی بود دسترسی به اکثر ...,f,مکان هتل آرامیس بسیار عالی‌بود دسترسی به اکثر ...,مکان هتل آرامیس بسیار عالی‌بود دسترسی به اکثر ...,مکان هتل آرامیس بسیار عالی‌بود دسترسی به اکثر ...,[مکان هتل آرامیس بسیار عالی‌بود دسترسی به اکثر...,"[مکان, هتل, آرامیس, بسیار, عالی‌بود, دسترسی, ب..."
3,/content/drive/MyDrive/NLP-Spring 99-00/HW1/tr...,اقا امیر می تونم بپرسم وضعیت غذای هتل از نظر ک...,f,اقا امیر می‌تونم بپرسم وضعیت غذای هتل از نظر ک...,اقا امیر می‌تونم بپرسم وضعیت غذای هتل از نظر ک...,اقا امیر می‌تونم بپرسم وضعیت غذای هتل از نظر ک...,[اقا امیر می‌تونم بپرسم وضعیت غذای هتل از نظر ...,"[اقا, امیر, می‌تونم, بپرسم, وضعیت, غذای, هتل, ..."
4,/content/drive/MyDrive/NLP-Spring 99-00/HW1/tr...,خب من ابن تابستون بودم و می خوام بگم خیلی بد ب...,f,خب من ابن تابستون‌بودم و می‌خوام بگم خیلی بد‌ب...,خب من ابن تابستون‌بودم و می‌خوام بگم خیلی بد‌ب...,خب من ابن تابستون‌بودم و می‌خوام بگم خیلی بد‌ب...,[خب من ابن تابستون‌بودم و می‌خوام بگم خیلی بد‌...,"[خب, من, ابن, تابستون‌بودم, و, می‌خوام, بگم, خ..."


Character Based Features

In [ ]:
#length of document
C = data['file_text'].apply(lambda text : len(text))

In [ ]:
data['F1'] = C

In [ ]:
def count_alphabet(text):
    count = 0
    for ch in text:
        if ch in alphabet:
            count = count + 1
    return count

In [ ]:
#number of alphabet characters in each document
data['F2'] = data['normalized_text'].apply(lambda text : count_alphabet(text)) / C

In [ ]:
def count_digits(text):
    count = 0
    for ch in text:
        if ch in numbers:
            count = count + 1
    return count

In [ ]:
#number of digital characters in each document
data['F3'] = data['normalized_text'].apply(lambda text : count_digits(text)) / C

In [ ]:
#number of white space characters in each document
data['F4'] = data['file_text'].apply(lambda text : text.count(" ")) / C

In [ ]:
#number of tab characters in each document
data['F5'] = data['file_text'].apply(lambda text : text.count("\t")) / C

In [ ]:
def count_special_chars(text):
    count = 0
    for ch in text:
        if ch in punctuation:
            count = count + 1
    return count

In [ ]:
#number of digital characters in each document
data['F6'] = data['file_text'].apply(lambda text : count_special_chars(text)) / C

Word Based Features

In [ ]:
#number of words in each document
N = data['tokenized_word'].apply(lambda lst : len(lst))
data['N'] = N

In [ ]:
data['F7'] = N

In [ ]:
#mean of words length in each document
data['F8'] = data['tokenized_word'].apply(lambda lst : statistics.mean(list(map(lambda x : len(x) , lst))))

In [ ]:
#number of unique words in each document
V = data['tokenized_word'].apply(lambda lst : len(set(lst)))
data['V'] = V
data['F9'] = V/N

In [ ]:
#number of words longer than 4 characters
data['F10'] = data['tokenized_word'].apply(lambda lst : sum(1 for x in lst if len(x) >= 5))/N

In [ ]:
#number of words shhorter than 4 characters
data['F11'] = data['tokenized_word'].apply(lambda lst : sum(1 for x in lst if len(x) <= 3))/N

In [ ]:
def get_word_by_count(list_of_words , count):
    dict_count = Counter(list_of_words)
    words = []
    for key,value in dict_count.items():
        if value == count:
            words.append(key)
    return len(words)

In [ ]:
#Hapax Legomena
data['F12'] = data['tokenized_word'].apply(lambda lst : get_word_by_count(lst , 1))/N

In [ ]:
#Hapax Dislegomena
data['F13'] = data['tokenized_word'].apply(lambda lst : get_word_by_count(lst , 2))/N

In [ ]:
def Yule_K(v,n,list_of_words):
    val = -1/n
    for i in range(1,v+1):
        val = val + get_word_by_count(list_of_words,i)*(i/n)**2
    val = val * 10**4
    return val

In [ ]:
def Simpsons(v,n,list_of_words):
    val = 0
    for i in range(1,v+1):
        try:
            val = val + get_word_by_count(list_of_words,i)*(i/n)*(i-1)/(n-1)
        except ZeroDivisionError:
            return None
    return val

In [ ]:
def Sichel(v,list_of_words):
    val = get_word_by_count(list_of_words,2)/v
    return val

In [ ]:
def Honores(v,n,list_of_words):
    try:
        val = 100 * (math.log10(n)) / (1 - get_word_by_count(list_of_words,1) / v)
    except ZeroDivisionError:
        return None
    return val

In [ ]:
data['F14'] = data.apply(lambda row : Yule_K(row['V'] , row['N'] , row['tokenized_word']) , axis=1)
data['F15'] = data.apply(lambda row : Simpsons(row['V'] , row['N'] , row['tokenized_word']) , axis=1)
data['F16'] = data.apply(lambda row : Sichel(row['V'] , row['tokenized_word']) , axis=1)
data['F17'] = data.apply(lambda row : Honores(row['V'] , row['N'] , row['tokenized_word']) , axis=1)

In [ ]:
data['F15'].interpolate(method ='linear', limit_direction ='forward' , inplace=True)
data['F17'].interpolate(method ='linear', limit_direction ='forward' , inplace=True)

In [ ]:
def entropy(list_of_words):
    dict_count = Counter(list_of_words)
    total = sum(dict_count.values())
    return sum(freq / total * log2(total / freq) for freq in dict_count.values())

data['F18'] = data['tokenized_word'].apply(lambda lst : entropy(lst))

In [ ]:
def get_count_by_len(list_of_words , length):
    count_list = list(map(lambda x : len(x) , list_of_words))
    dict_count = Counter(count_list)
    return dict_count[length]

# word length histogram
#init F19 to F38
for i in range(1,21):
    feature_name = "F" + str(i + 18)
    data[feature_name] = data['tokenized_word'].apply(lambda lst : get_count_by_len(lst , i))

In [ ]:
with open(config['current_working_dir'] + 'Lexicon_Positive.pickle', 'rb') as file:
    lexicon_positive = [normalizer.normalize(x) for x in pickle.load(file)]

In [ ]:
#number of positive words in each document
data['F39'] = data['tokenized_word'].apply(lambda lst : sum(1 for x in lst if x in lexicon_positive))

In [ ]:
with open(config['current_working_dir'] + 'Lexicon_Negative.pickle', 'rb') as file:
    lexicon_negative = [normalizer.normalize(x) for x in pickle.load(file)]

In [ ]:
#number of negative words in each document
data['F40'] = data['tokenized_word'].apply(lambda lst : sum(1 for x in lst if x in lexicon_negative))

In [ ]:
with open(config['current_working_dir'] + 'Lexicon_Anger.pickle', 'rb') as file:
    lexicon_anger = [normalizer.normalize(x) for x in pickle.load(file)]

In [ ]:
#number of anger words in each document
data['F41'] = data['tokenized_word'].apply(lambda lst : sum(1 for x in lst if x in lexicon_anger))

In [ ]:
with open(config['current_working_dir'] + 'Lexicon_Anticipation.pickle', 'rb') as file:
    lexicon_anticipation = [normalizer.normalize(x) for x in pickle.load(file)]

In [ ]:
#number of anticipation words in each document
data['F42'] = data['tokenized_word'].apply(lambda lst : sum(1 for x in lst if x in lexicon_anticipation))

In [ ]:
with open(config['current_working_dir'] + 'Lexicon_Disgust.pickle', 'rb') as file:
    lexicon_disgust = [normalizer.normalize(x) for x in pickle.load(file)]

In [ ]:
#number of disgusting words in each document
data['F43'] = data['tokenized_word'].apply(lambda lst : sum(1 for x in lst if x in lexicon_disgust))

In [ ]:
with open(config['current_working_dir'] + 'Lexicon_Fear.pickle', 'rb') as file:
    lexicon_fear = [normalizer.normalize(x) for x in pickle.load(file)]

In [ ]:
#number of fearnig words in each document
data['F44'] = data['tokenized_word'].apply(lambda lst : sum(1 for x in lst if x in lexicon_fear))

In [ ]:
with open(config['current_working_dir'] + 'Lexicon_Joy.pickle', 'rb') as file:
    lexicon_joy = [normalizer.normalize(x) for x in pickle.load(file)]

In [ ]:
#number of joy words in each document
data['F45'] = data['tokenized_word'].apply(lambda lst : sum(1 for x in lst if x in lexicon_joy))

In [ ]:
with open(config['current_working_dir'] + 'Lexicon_Sadness.pickle', 'rb') as file:
    lexicon_sadness = [normalizer.normalize(x) for x in pickle.load(file)]

In [ ]:
#number of sad words in each document
data['F46'] = data['tokenized_word'].apply(lambda lst : sum(1 for x in lst if x in lexicon_sadness))

In [ ]:
with open(config['current_working_dir'] + 'Lexicon_Surprise.pickle', 'rb') as file:
    lexicon_surprise = [normalizer.normalize(x) for x in pickle.load(file)]

In [ ]:
#number of surprising words in each document
data['F47'] = data['tokenized_word'].apply(lambda lst : sum(1 for x in lst if x in lexicon_surprise))

In [ ]:
with open(config['current_working_dir'] + 'Lexicon_Trust.pickle', 'rb') as file:
    lexicon_trust = [normalizer.normalize(x) for x in pickle.load(file)]

In [ ]:
#number of trusting words in each document
data['F48'] = data['tokenized_word'].apply(lambda lst : sum(1 for x in lst if x in lexicon_trust))

Syntax Based Features

In [ ]:
data['F49'] = data['normalized_text'].apply(lambda text : text.count("'"))/C

In [ ]:
data['F50'] = data['normalized_text'].apply(lambda text : text.count("،"))/C

In [ ]:
data['F51'] = data['normalized_text'].apply(lambda text : text.count("."))/C

In [ ]:
data['F52'] = data['normalized_text'].apply(lambda text : text.count(":"))/C

In [ ]:
data['F53'] = data['normalized_text'].apply(lambda text : text.count(";"))/C

In [ ]:
data['F54'] = data['normalized_text'].apply(lambda text : text.count("؟"))/C

In [ ]:
data['F55'] = data['normalized_text'].apply(lambda text : len(re.findall("؟؟+" , text)))/C

In [ ]:
data['F56'] = data['normalized_text'].apply(lambda text : text.count("!"))/C

In [ ]:
data['F57'] = data['normalized_text'].apply(lambda text : len(re.findall("!!+" , text)))/C

In [ ]:
data['F58'] = data['normalized_text'].apply(lambda text : len(re.findall("(\.)(\.)+" , text)))/C

Structural Features

In [ ]:
#number of document lines
data['F59'] = data['file_text'].apply(lambda text : text.count("\n"))

In [ ]:
#number of sentences in each document
S = data['tokenized_sentence'].apply(lambda lst : len(lst))
data['F60'] = S

In [ ]:
#average number of words in each sentence
data['F61'] = data['tokenized_sentence'].apply(lambda lst : statistics.mean(list(map(lambda x : len(tokenizer.tokenize_words(x)) , lst))))

In [ ]:
def ratio_of_empty_lines(text):
    all_lines = text.split("\n")
    number_of_empty_lines = 0
    for line in all_lines:
        if len(line.strip()) == 0:
            number_of_empty_lines = number_of_empty_lines + 1
    return number_of_empty_lines / len(all_lines)

In [ ]:
data['F62'] = data['file_text'].apply(lambda text : ratio_of_empty_lines(text))

In [ ]:
def avg_length_of_nonempty_lines(text):
    all_lines = text.split("\n")
    length_of_non_empty_lines = 0
    number_of_non_empty_lines = 0
    for line in all_lines:
        if len(line.strip()) != 0:
            number_of_non_empty_lines = number_of_non_empty_lines + 1
            length_of_non_empty_lines = length_of_non_empty_lines + len(line)
    try:
        return length_of_non_empty_lines / number_of_non_empty_lines
    except ZeroDivisionError:
        return 0

In [ ]:
data['F63'] = data['file_text'].apply(lambda text : avg_length_of_nonempty_lines(text))

Grammer Related Features

In [ ]:
#gender specific words
female_words = ["شوهر" , "شوهرم"]
male_words = ["خانمم" , "زنم"]
data['F64'] = data['cleaned_punc_text'].apply(lambda text : sum([1 for word in female_words if word in text])).astype(int)
data['F65'] = data['cleaned_punc_text'].apply(lambda text : sum([1 for word in male_words if word in text])).astype(int)

In [ ]:
with open(config['current_working_dir'] + 'interrogative.pickle', 'rb') as file:
    list_of_all_interrogatives = pickle.load(file)

In [ ]:
data['F66'] = data['tokenized_word'].apply(lambda lst : sum(1 for x in lst if x in list_of_all_interrogatives))/N

In [ ]:
with open(config['current_working_dir'] + 'conjunctions.pickle', 'rb') as file:
    list_of_all_conjunctions = pickle.load(file)

In [ ]:
data['F67'] = data['tokenized_word'].apply(lambda lst : sum(1 for x in lst if x in list_of_all_conjunctions))/N

In [ ]:
with open(config['current_working_dir'] + 'interjections.pickle', 'rb') as file:
    list_of_all_interjections = pickle.load(file)

In [ ]:
data['F68'] = data['tokenized_word'].apply(lambda lst : sum(1 for x in lst if x in list_of_all_interjections))/N

In [ ]:
#Part Of Speech Tagger
tags_path = config['current_working_dir'] + 'tags.csv'
if os.path.isfile(tags_path):
    tags = pd.read_csv(tags_path)
else:
    tags = pd.Series()
    for index, row in data.iterrows():
        #delete tagger and re initialize it on every 20 iterations for avoiding java head error
        if index % 20 == 0:
            del tagger
            tagger = POSTagger(tagging_model="stanford")

        clear_output()
        print(str(index))
        tags = tags.append(pd.Series([tagger.parse(row['tokenized_word'])]), ignore_index=True)
    tags.to_csv(tags_path , index=False)
    tags = pd.read_csv(tags_path)

In [ ]:
#number of propositions in each document
data['F69'] = tags.iloc[: , 0].apply(lambda lst : sum(1 for tag in list(ast.literal_eval(lst)) if tag[1] == 'PO')) / N

Linguistic-Psychological Features

In [ ]:
polarity_df = pd.read_csv(config['current_working_dir'] + 'PersianSWN.csv' , sep='\t' , names = ["id" , "word" , "conf" , "plus" , "minus"] ,header=None)
polarity_df.drop("id" , axis=1 , inplace=True)
polarity_df["plus_prob"] = polarity_df["conf"] * polarity_df["plus"]
polarity_df["minus_prob"] = polarity_df["conf"] * polarity_df["minus"]
polarity_df.drop("plus" , axis=1 , inplace=True)
polarity_df.drop("minus" , axis=1 , inplace=True)

In [ ]:
ploar_plus = polarity_df.groupby("word")["plus_prob"].mean().reset_index()
ploar_minus = polarity_df.groupby("word")["minus_prob"].mean().reset_index()
word_polatiry = pd.merge(ploar_plus , ploar_minus , how="inner" , on="word")

In [ ]:
good_words = word_polatiry[(word_polatiry["plus_prob"] > word_polatiry["minus_prob"] + 0.15) & (word_polatiry["plus_prob"] > 0.2)]["word"].unique()
bad_words = word_polatiry[(word_polatiry["minus_prob"] > word_polatiry["plus_prob"] + 0.15) & (word_polatiry["minus_prob"] > 0.2)]["word"].unique()

In [ ]:
good_words = list(map(lambda x : normalizer.normalize(x) , good_words))
bad_words = list(map(lambda x : normalizer.normalize(x) , bad_words))

In [ ]:
data['F70'] = data['tokenized_word'].apply(lambda lst : sum(1 for x in lst if x in good_words))

In [ ]:
data['F71'] = data['tokenized_word'].apply(lambda lst : sum(1 for x in lst if x in bad_words))

In [ ]:
with open(config['current_working_dir'] + 'colors.pickle', 'rb') as file:
    list_of_all_colors = pickle.load(file)

In [ ]:
#number of colors in each document
data['F72'] = data['tokenized_word'].apply(lambda lst : sum(1 for x in lst if x in list_of_all_colors))

In [ ]:
#number of adjectives in each document
data['F73'] = tags.iloc[: , 0].apply(lambda lst : sum(1 for tag in list(ast.literal_eval(lst)) if tag[1] == 'ADJ'))

In [ ]:
#number of adverbs in each document
data['F74'] = tags.iloc[: , 0].apply(lambda lst : sum(1 for tag in list(ast.literal_eval(lst)) if tag[1] == 'ADV'))

In [ ]:
#number of pronouns in each document
data['F75'] = tags.iloc[: , 0].apply(lambda lst : sum(1 for tag in list(ast.literal_eval(lst)) if tag[1] == 'PRO'))

In [ ]:
with open(config['current_working_dir'] + 'doubt.pickle', 'rb') as file:
    list_of_all_doubt = pickle.load(file)

In [ ]:
#number of doubt words in each document
data['F76'] = data['tokenized_word'].apply(lambda lst : sum(1 for x in lst if x in list_of_all_doubt))

In [ ]:
with open(config['current_working_dir'] + 'certainty.pickle', 'rb') as file:
    list_of_all_certainties = pickle.load(file)

In [ ]:
#number of certainty words in each document
data['F77'] = data['tokenized_word'].apply(lambda lst : sum(1 for x in lst if x in list_of_all_certainties))

In [ ]:
#greeting
greetings = ["سلام" , "خوبید"]
data['F78'] = data['cleaned_punc_text'].apply(lambda text : sum([1 for word in greetings if word in text]) >= 1).astype(int)

In [ ]:
#farewell
farewells = ["خداحافظ" , "خدانگهدار"]
data['F79'] = data['cleaned_punc_text'].apply(lambda text : sum([1 for word in farewells if word in text]) >= 1).astype(int)

In [ ]:
data.tail()

,file_path,file_text,gender,normalized_text,cleaned_number_text,cleaned_punc_text,tokenized_sentence,tokenized_word,F1,F2,F3,F4,F5,F6,N,F7,F8,V,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28,F29,F30,...,F40,F41,F42,F43,F44,F45,F46,F47,F48,F49,F50,F51,F52,F53,F54,F55,F56,F57,F58,F59,F60,F61,F62,F63,F64,F65,F66,F67,F68,F69,F70,F71,F72,F73,F74,F75,F76,F77,F78,F79
1995,/content/drive/MyDrive/NLP-Spring 99-00/HW1/tr...,سلام من در 24 شهریوره 1390به مدت سه شب به همرا...,m,سلام من در 24 شهریوره 1390به مدت سه شب به همرا...,سلام من در شهریوره به مدت سه شب به همراه خانو...,سلام من در شهریوره به مدت سه شب به همراه خانو...,[سلام من در 24 شهریوره 1390به مدت سه شب به همر...,"[سلام, من, در, شهریوره, به, مدت, سه, شب, به, ه...",332,0.626506,0.018072,0.334337,0.0,0.018072,54,54,3.962963,44,0.814815,0.388889,0.481481,0.722222,0.037037,116.598080,0.011880,0.045455,1524.506509,5.296464,2,15,9,7,10,3,6,1,1,0,0,0,...,1,0,4,1,0,4,0,1,3,0.0,0.000000,0.018072,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,1,6,10.166667,0.500000,316.000000,0,0,0.000000,0.037037,0.0,0.166667,0,0,0,5,2,3,0,0,1,0
1996,/content/drive/MyDrive/NLP-Spring 99-00/HW1/tr...,مابه اتفاق پدرومادرمان درتاریخ 7 تا10 مرداد 90...,m,مابه اتفاق پدرومادرمان درتاریخ 7 تا10 مرداد 90...,مابه اتفاق پدرومادرمان درتاریخ تا مرداد دراین...,مابه اتفاق پدرومادرمان درتاریخ تا مرداد دراین...,[مابه اتفاق پدرومادرمان درتاریخ 7 تا10 مرداد 9...,"[مابه, اتفاق, پدرومادرمان, درتاریخ, تا, مرداد,...",288,0.684028,0.017361,0.270833,0.0,0.024306,44,44,4.545455,39,0.886364,0.500000,0.340909,0.818182,0.045455,82.644628,0.008457,0.051282,2136.488479,5.186704,0,7,8,7,9,7,4,1,0,0,1,0,...,0,0,3,0,0,1,0,3,2,0.0,0.000000,0.010417,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,1,3,17.000000,0.500000,272.000000,0,0,0.000000,0.045455,0.0,0.159091,0,0,0,5,1,0,0,0,0,0
1997,/content/drive/MyDrive/NLP-Spring 99-00/HW1/tr...,وضعیت اتاقها مناسب و تمیز بود ، ولی در حد هتل ...,m,وضعیت اتاقها مناسب و تمیز‌بود ، ولی در حد هتل ...,وضعیت اتاقها مناسب و تمیز‌بود ، ولی در حد هتل ...,وضعیت اتاقها مناسب و تمیز‌بود ، ولی در حد هتل ...,[وضعیت اتاقها مناسب و تمیز‌بود ، ولی در حد هتل...,"[وضعیت, اتاقها, مناسب, و, تمیز‌بود, ،, ولی, در...",238,0.659664,0.008403,0.298319,0.0,0.008403,39,39,4.128205,30,0.769231,0.435897,0.384615,0.589744,0.128205,144.641683,0.014845,0.166667,681.884832,4.785152,4,4,7,7,10,2,2,3,0,0,0,0,...,2,1,6,0,1,6,0,2,6,0.0,0.004202,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,3,3,13.666667,0.250000,73.333333,0,0,0.000000,0.128205,0.0,0.076923,0,0,0,6,2,0,0,0,0,0
1998,/content/drive/MyDrive/NLP-Spring 99-00/HW1/tr...,با سلام\nمن در اغلب هتل های 5 ستاره کیش اقامت ...,m,با سلام\nمن در اغلب هتل‌های 5 ستاره کیش اقامت ...,با سلام\nمن در اغلب هتل‌های ستاره کیش اقامت د...,با سلام\nمن در اغلب هتل‌های ستاره کیش اقامت د...,"[با سلام , من در اغلب هتل‌های 5 ستاره کیش اقام...","[با, سلام, من, در, اغلب, هتل‌های, ستاره, کیش, ...",1777,0.737198,0.010129,0.221159,0.0,0.020822,344,344,3.875000,193,0.561047,0.351744,0.456395,0.418605,0.069767,98.363981,0.009865,0.124352,999.093427,7.032848,15,76,66,66,61,31,21,6,2,0,0,0,...,16,4,21,3,3,19,7,5,26,0.0,0.000000,0.007316,0.001688,0.0,0.0,0.0,0.0,0.0,0.000563,18,16,24.750000,0.210526,116.266667,0,0,0.040698,0.101744,0.0,0.107558,2,0,2,25,18,6,0,1,1,0
1999,/content/drive/MyDrive/NLP-Spring 99-00/HW1/tr...,من 4 مرتبه در هتل پارمیس اقامت داشتم از همه نظ...,m,من 4 مرتبه در هتل پارمیس اقامت داشتم از همه نظ...,من مرتبه در هتل پارمیس اقامت داشتم از همه نظر...,من مرتبه در هتل پارمیس اقامت داشتم از همه نظر...,[من 4 مرتبه در هتل پارمیس اقامت داشتم از همه ن...,"[من, مرتبه, در, هتل, پارمیس, اقامت, داشتم, از,...",192,0.635417,0.005208,0.322917,0.0,0.015625,31,31,4.032258,27,0.870968,0.451613,0.419355,0.741935,0.129032,83.246618,0.008602,0.148148,1006.669143,4.696132,2,6,5,4,7,5,1,1,0,0,0,0,...,1,1,2,0,2,3,0,0,1,0.0,0.010417,0.015625,0.000000,0.0,0.0,0.0,0.0,0.0,0.005208,2,3,11.000000,0.333333,87.500000,0,0,0.000000,0.000000,0.0,0.161290,0,0,0,3,2,1,0,0,0,0


**Classification**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import sklearn.preprocessing as preprocessing
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
import sklearn.tree as tree
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.utils import shuffle
from sklearn import svm
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import VarianceThreshold
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline

Add word2vec features

In [ ]:
model = gensim.models.KeyedVectors.load_word2vec_format(config["current_working_dir"] + "blog.vec", binary=False)

In [ ]:
lst = []
for index, row in data.iterrows():
    lst_i = []
    for x in row['tokenized_word']:
        if x in model.vocab:
            lst_i.append(model.get_vector(x))
    lst.append(np.mean(lst_i , axis=0).tolist())

In [ ]:
df_w2v = pd.DataFrame(lst , columns=[("W" + str(i)) for i in range(1,301)])

In [ ]:
df_w2v.head()

,W1,W2,W3,W4,W5,W6,W7,W8,W9,W10,W11,W12,W13,W14,W15,W16,W17,W18,W19,W20,W21,W22,W23,W24,W25,W26,W27,W28,W29,W30,W31,W32,W33,W34,W35,W36,W37,W38,W39,W40,...,W261,W262,W263,W264,W265,W266,W267,W268,W269,W270,W271,W272,W273,W274,W275,W276,W277,W278,W279,W280,W281,W282,W283,W284,W285,W286,W287,W288,W289,W290,W291,W292,W293,W294,W295,W296,W297,W298,W299,W300
0,-0.635701,0.404710,0.232618,0.649962,-0.559817,0.104589,-0.694853,-0.368469,0.466647,0.763832,0.361236,-0.405468,-0.408364,-0.490691,-1.167635,-0.870573,0.431136,-0.948014,0.344362,0.179712,-0.240547,-0.323280,-0.109500,0.071483,0.261787,0.378309,0.027454,0.032446,-0.799006,0.558962,1.121131,-0.860536,0.210647,0.670594,0.228576,-0.176516,-0.052338,0.198432,-0.578279,-0.381580,...,0.657024,0.004690,-0.368087,-0.794081,0.187754,-0.076919,-0.841035,-0.202321,-0.435882,0.256242,-0.235725,-0.274914,-0.197347,-0.344894,-0.012548,-0.618689,-0.406323,-0.292166,-0.503344,-0.364123,0.077835,-0.235606,0.373729,-0.333452,0.326518,0.617376,-0.096838,-0.731296,0.931937,-0.085088,0.340682,-0.124236,1.050792,-0.257425,0.286342,-0.387435,0.037284,0.083712,-1.034085,-0.628029
1,-1.412256,0.037336,0.361358,0.263031,-0.150270,-0.018799,-0.086218,-0.294520,0.293137,-0.001957,0.000868,-0.604539,-0.711037,-0.398851,-1.142186,-0.556385,0.373352,-0.805543,-0.089147,0.103626,0.004238,0.276039,-0.132823,-0.159927,0.716809,0.412252,0.174666,0.181588,-0.619710,0.602582,1.728862,-0.564585,-0.277225,0.384372,-0.040445,-0.050157,-0.119540,0.195473,-0.027744,-0.310011,...,0.217190,-0.433596,-0.306056,-0.963629,0.150714,-0.344677,-0.298506,-0.343407,-0.226154,0.502750,-0.682157,0.298864,-0.487338,0.218779,-0.496535,-0.537937,-0.720525,-0.091531,-0.341946,-0.920971,0.140072,-0.050491,0.049839,-0.694503,0.313349,-0.286679,-0.951356,-0.256524,0.411284,-0.203807,-0.014366,0.026964,0.475904,0.268695,0.222958,-0.243880,0.404042,0.184965,-1.036200,-0.403382
2,-0.826628,0.378644,-0.378016,0.812562,0.025193,-0.166233,0.434905,-0.668231,0.076106,-0.319143,-0.677103,-0.315311,-0.382423,0.133176,-0.552535,-0.900588,-0.027599,-0.343373,-0.368302,-0.204806,0.119711,0.083379,0.079011,-0.198031,0.632295,-0.238068,-0.121024,0.240585,-0.247756,0.149469,0.993158,-1.119681,0.143618,0.244408,-0.126841,-0.456586,0.873347,-0.299515,0.242541,-0.450768,...,-0.326163,-0.082648,0.864174,0.025726,-0.143014,0.137354,0.003033,0.604713,-0.439455,-0.305126,-0.479047,-0.107936,0.208066,-0.104466,-0.276986,0.203378,-0.605908,0.173829,0.136090,0.529243,0.304045,-0.031962,-0.592082,-0.248970,0.388911,-0.334840,-0.218329,0.292355,0.237939,0.111456,0.100229,-0.130652,0.562838,0.514301,0.118097,0.200222,-0.381025,-0.759330,-0.161188,0.283169
3,-0.286528,0.839094,0.273756,0.529813,-0.136855,0.221745,-0.993268,-0.700296,0.987354,1.103176,-0.218767,-0.956841,-0.266598,-0.574656,-0.474153,-0.194772,0.193493,-1.678120,1.144959,-0.053987,-1.149172,0.246702,0.165669,-0.045313,1.001426,-0.412041,0.779227,-0.872738,-0.840521,1.107864,1.917972,-1.231302,0.851690,1.679757,0.982163,-0.259702,1.130288,-0.747177,-1.155273,-0.779009,...,0.116572,-0.010849,0.260437,-0.412125,-0.029503,-0.489302,-0.930571,0.307329,0.535768,0.675746,-0.535199,-1.143132,-0.294593,-1.221241,-0.518914,-0.527300,0.042593,-1.210058,-0.700622,-0.201417,0.128154,0.195513,-1.035723,-0.148723,-0.095904,0.831423,-0.594786,-1.094048,0.887264,0.033080,0.796383,0.656505,2.013035,-0.276449,1.078734,-0.280497,0.990778,-1.350121,0.119359,-0.512164
4,-0.588570,0.193512,0.360910,0.583438,0.221490,-0.380701,0.005004,-0.257226,1.079256,1.799947,0.924396,-0.223107,-0.621502,-0.159034,-1.417933,-0.670448,0.257183,-1.415246,0.248792,0.082706,-0.364851,0.072303,-0.374240,0.013575,-0.031482,0.684079,-0.093236,-0.505219,-0.792044,0.858887,1.319287,-0.501601,0.214128,0.207913,0.327486,-0.578092,-0.435174,0.025519,-0.210221,-0.968580,...,0.205118,-0.096458,0.132171,-0.443417,0.404171,0.157335,-0.929505,-0.080740,-0.102321,0.570660,0.185611,0.194830,-0.287876,-0.295257,0.033552,-0.619817,0.092701,-1.115365,-0.

In [ ]:
data_total = pd.concat([data, df_w2v], axis=1)

In [ ]:
x = data_total.filter(axis=1 , regex="[F+W].*")

y = data['gender'].copy()
y[y == 'f'] = 0
y[y == 'm'] = 1
y=y.astype('int')

rs = 4

In [ ]:
x , y =  shuffle(x, y , random_state=rs)

In [ ]:
x.tail()

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28,F29,F30,F31,F32,F33,F34,F35,F36,F37,F38,F39,F40,...,W261,W262,W263,W264,W265,W266,W267,W268,W269,W270,W271,W272,W273,W274,W275,W276,W277,W278,W279,W280,W281,W282,W283,W284,W285,W286,W287,W288,W289,W290,W291,W292,W293,W294,W295,W296,W297,W298,W299,W300
1921,1016,0.769685,0.007874,0.198819,0.0,0.010827,201,3.960199,0.781095,0.368159,0.457711,0.661692,0.079602,46.038464,0.004627,0.101911,1506.674088,7.082491,4,39,49,35,39,21,8,5,0,1,0,0,0,0,0,0,0,0,0,0,22,10,...,0.360300,0.067244,-0.024090,-0.735086,0.229458,-0.082572,-0.812864,-0.016908,-0.194577,0.679891,-0.368325,-0.180630,-0.268921,-0.200243,-0.224576,-0.345331,0.275874,-0.210232,-0.214703,-0.619071,0.369032,-0.588195,0.312292,0.186018,0.606119,0.732794,-0.824051,-1.151310,0.203969,-0.273627,0.157013,0.174499,0.915715,0.031888,0.385749,-0.781162,0.189649,-0.217080,-0.366993,-0.992995
709,1285,0.758755,0.000000,0.224903,0.0,0.007004,251,3.936255,0.637450,0.330677,0.446215,0.513944,0.063745,96.506405,0.009689,0.100000,1238.541276,6.860341,11,56,45,56,41,20,13,4,1,0,1,2,0,1,0,0,0,0,0,0,41,4,...,-0.407994,0.020538,0.548861,-0.246781,0.163027,0.046224,0.063885,0.161552,-0.156271,0.296376,-0.302397,-0.316428,-0.008961,-0.002094,-0.703987,0.366364,-0.565550,0.197509,-0.031963,-0.330613,0.564899,-0.286578,-0.735598,-0.615951,0.213052,-0.226020,0.285604,0.271071,0.669928,-0.119319,0.020339,-0.257207,0.667381,0.443523,-0.032346,-0.164977,-0.495420,-0.555300,-0.980773,0.413193
439,834,0.766187,0.005995,0.217026,0.0,0.004796,166,3.909639,0.686747,0.319277,0.481928,0.548193,0.084337,124.836696,0.012559,0.122807,1100.401400,6.457257,15,32,33,33,16,16,12,6,2,0,0,0,1,0,0,0,0,0,0,0,11,10,...,-0.184604,-0.162306,0.282334,-0.072941,-0.133716,0.234327,-0.184311,-0.034659,-0.044534,0.340242,-0.303425,-0.170005,-0.147301,-0.133485,-0.233053,0.085187,-0.323400,-0.041915,-0.120834,-0.323193,-0.148574,0.071984,-0.300826,-0.039210,0.031099,0.419009,-0.443828,-0.437741,-0.251363,-0.064134,-0.103397,0.019121,0.542171,0.124715,0.416749,-0.137232,-0.310769,0.075754,-0.607480,-0.736259
174,283,0.724382,0.000000,0.233216,0.0,0.000000,55,4.000000,0.763636,0.254545,0.454545,0.600000,0.145455,152.066116,0.015488,0.190476,812.169255,5.208455,2,6,17,16,5,3,1,4,1,0,0,0,0,0,0,0,0,0,0,0,3,0,...,0.240980,-0.294585,0.447230,-1.221809,0.051780,0.301582,-0.418016,-0.279703,-0.358681,0.528461,-0.671412,0.082269,0.005381,0.220199,-0.216347,-0.211819,-0.414028,0.320501,0.207835,-0.499359,-0.713960,0.152257,-0.298310,-0.042461,0.113320,0.732711,-0.911356,-1.032955,0.275314,-0.459419,-0.313432,0.066945,0.810087,0.143098,0.201602,0.199210,-0.150127,0.832191,-1.087860,-1.195407
1146,286,0.720280,0.031469,0.213287,0.0,0.034965,53,3.943396,0.849057,0.358491,0.396226,0.754717,0.075472,99.679601,0.010160,0.088889,1551.848283,5.357927,5,7,9,13,11,1,6,1,0,0,0,0,0,0,0,0,0,0,0,0,10,2,...,-0.049853,-0.188264,0.016939,-0.665145,0.070591,0.024407,-0.194316,0.485244,-0.077366,-0.131398,-0.250792,-0.345567,0.002656,-0.186676,-0.433098,-0.106782,0.011263,-0.411976,-0.188139,-0.328999,-0.154016,-0.182251,-0.464097,0.291674,0.589224,0.050517,0.043899,0.239812,0.391236,-0.008862,-0.310891,-0.614316,1.072485,0.499896,0.395544,-0.115520,-0.062552,-0.608668,-0.626549,0.133914


In [ ]:
clf = svm.SVC(gamma=0.4)
kf = KFold(n_splits=config['k'])
accuracy = 0
for train_index, test_index in kf.split(x):
    x_train, x_test = x.iloc[train_index], x.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    #transform all feature's values to range [0 1]
    min_max_scaler = preprocessing.MinMaxScaler()
    min_max_scaler.fit(x_train)
    x_train = min_max_scaler.transform(x_train)
    x_test = min_max_scaler.transform(x_test)

    variance_threshold = VarianceThreshold(threshold=(0.004))
    variance_threshold.fit(x_train)
    x_train = variance_threshold.transform(x_train)
    x_test = variance_threshold.transform(x_test)
        
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    accuracy = accuracy + accuracy_score(y_test, y_pred)
print(accuracy / 10)

0.6385
